In [1]:
# Optimize Portfolio VaR by Adjusting Composition of MSFT and GOOG in the portfolio


import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import numpy as np

# Define the stock symbols and the period
symbols = ['GOOG', 'MSFT', 'SPY']
period = '5y'

# Create an empty DataFrame to store the closing prices
data = pd.DataFrame()

# Fetch the closing prices for each stock
for symbol in symbols:
    data[symbol] = yf.Ticker(symbol).history(period=period)['Close']

# Normalize the prices so they all start at $1
normalized_data = data / data.iloc[0] * 1

# Calculate daily returns for each stock
returns = data.pct_change().dropna()

# Calculate Value at Risk (VaR) for GOOG and MSFT
confidence_level = 0.95  # 95% confidence level
VaR = {}

for symbol in ['GOOG', 'MSFT']:
    # Calculate the daily VaR
    daily_returns = returns[symbol]
    VaR[symbol] = np.percentile(daily_returns, (1 - confidence_level) * 100)

# Function to calculate Portfolio VaR
def portfolio_var(weights):
    portfolio_returns = returns[['GOOG', 'MSFT']].dot(weights)
    portfolio_VaR = np.percentile(np.abs(portfolio_returns), (1 - confidence_level) * 100)
    return portfolio_VaR

# Calculate VaR for different ratios of MSFT and GOOG
ratios = np.arange(0, 1.1, 0.1)
portfolio_VaRs = []

for ratio in ratios:
    weights = [ratio, 1 - ratio]
    portfolio_VaR = portfolio_var(weights)
    portfolio_VaRs.append((weights, portfolio_VaR))
    
# Find the ratio with minimum absolute VaR
min_var_ratio = min(portfolio_VaRs, key=lambda x: x[1])

# Print the results
print("\nMSFT:GOOG Ratio and Portfolio VaR at 95% confidence level:")
for weights, portfolio_VaR in portfolio_VaRs:
    print(f"MSFT: {weights[0]:.1f}, GOOG: {weights[1]:.1f} -> Portfolio VaR: {portfolio_VaR * 100:.2f}%")

# Print the ratio with minimum absolute VaR
print(f"\nMinimum Absolute Portfolio VaR is at MSFT: {min_var_ratio[0][0]:.1f}, GOOG: {min_var_ratio[0][1]:.1f} with VaR: {min_var_ratio[1] * 100:.2f}%")


MSFT:GOOG Ratio and Portfolio VaR at 95% confidence level:
MSFT: 0.0, GOOG: 1.0 -> Portfolio VaR: 0.09%
MSFT: 0.1, GOOG: 0.9 -> Portfolio VaR: 0.07%
MSFT: 0.2, GOOG: 0.8 -> Portfolio VaR: 0.07%
MSFT: 0.3, GOOG: 0.7 -> Portfolio VaR: 0.07%
MSFT: 0.4, GOOG: 0.6 -> Portfolio VaR: 0.07%
MSFT: 0.5, GOOG: 0.5 -> Portfolio VaR: 0.08%
MSFT: 0.6, GOOG: 0.4 -> Portfolio VaR: 0.09%
MSFT: 0.7, GOOG: 0.3 -> Portfolio VaR: 0.09%
MSFT: 0.8, GOOG: 0.2 -> Portfolio VaR: 0.10%
MSFT: 0.9, GOOG: 0.1 -> Portfolio VaR: 0.08%
MSFT: 1.0, GOOG: 0.0 -> Portfolio VaR: 0.08%

Minimum Absolute Portfolio VaR is at MSFT: 0.4, GOOG: 0.6 with VaR: 0.07%
